# Script for crafting Adversarial Examples

In [1]:
# get data from important word detector
%store -r important_words
%store -r sentence_packages

## Methed 2: Mispeeling

#### 1. Create modified Dataset

In [ ]:
original_sentences = []
for sentence in sentence_packages:
    original_sentences.append(sentence['original_sentence'])

In [2]:
Das erste Typo Wort wird verwendet..

1 über den txt birkbeck corpera iterieren und die Wörter suchen
die mit $ beginnen.

2. diese und das Wort direkt danach in ein dict speichern

3. über die important words gehen und cecken, ob sie im dict vorkommen.

4. wenn ja, dann speichern in list modified words

In [28]:
with open('birkbeck_corpera_typo.txt', 'r') as f:
    typods = f.readlines()
    
typodict = {}
counter = 0
for item in typods:
    if typods[counter][0] == '$':
        word = typods[counter].strip('\n').strip('$')
        typo = typods[counter+1].strip('\n')
        typodict.update({word : typo})
    counter += 1

In [ ]:
modified_words = []
def typo(word):
    for word in important_words:
        if word in typodict:
            modified_words.append(typodict[])
        
        

    return ''.join(getchar(c) for c in text)

In [12]:
modified_words = []
for word in important_words:
    modified_words.append(typo(word))

In [13]:
modified_sentences = []
index = 0
for sentence in original_sentences:
    modified_sentences.append(sentence.replace(important_words[index], modified_words[index]))
    index += 1

#### 2. Import BERT Model

In [14]:
import torch
import random
import numpy as np
import pandas as pd

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [15]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

In [16]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [17]:
# prediciton function
def predict_sentiment(sentence):
    model.eval()
    inputs = tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors='pt')
    prediction = model(inputs['input_ids'], token_type_ids=inputs['token_type_ids'])[0].argmax().item()
    return prediction

#### 3. Prediction

In [18]:
original_predictions = []
for sentence in original_sentences:
    original_predictions.append(predict_sentiment(sentence))   

In [19]:
modified_predictions = []
for sentence in modified_sentences:
    modified_predictions.append(predict_sentiment(sentence))

#### 4. Creation of Dataset

In [21]:
ds = pd.DataFrame(list(zip(original_sentences, original_predictions, modified_sentences, modified_predictions)),
                 columns = ['original_sentences', 'original_predictions', 'modified_sentences', 'modified_predictions'])

In [23]:
ds.head()

,original_sentences,original_predictions,modified_sentences,modified_predictions
0,I visited this b&b during a short trip to ride...,4,I vi5it3d this b&b during a short trip to ride...,4
1,"The owners were really lovely people, the room...",4,"The owners w3r3 really lovely people, the room...",4
2,It is in a really good location for riding or ...,3,It is in a really good location for riding or ...,3
3,I cannot recommend this place enough!.,0,I cannot recommend this place 3n0ugh!.,0
4,3 friends and I visited 't Materke in April 20...,4,3 friends and I visited 't Materke in April 20...,4


In [24]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   original_sentences    436 non-null    object
 1   original_predictions  436 non-null    int64 
 2   modified_sentences    436 non-null    object
 3   modified_predictions  436 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 13.8+ KB


#### 5. Creation of adversarial Dataset

In [27]:
advds = ds[ds.original_predictions != ds.modified_predictions]
advds.head()

,original_sentences,original_predictions,modified_sentences,modified_predictions
19,"It says it is a hotel, but actually nothing mo...",1,"It 54y5 it is a hotel, but actually nothing mo...",2
26,I went to this hotel because I could not find ...,2,I w3nt to this hotel because I could not find ...,4
30,"Rooms are simple, bathrooms adequate.",2,"Rooms are simple, bathrooms 4d3qu4t3.",3
33,Just outside but close to city center & has an...,3,Just 0ut5id3 but close to city center & has an...,2
39,Situated right across the castle with a fabulo...,4,Situated right across the castle with a f4bu10...,3


In [28]:
advds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83 entries, 19 to 428
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   original_sentences    83 non-null     object
 1   original_predictions  83 non-null     int64 
 2   modified_sentences    83 non-null     object
 3   modified_predictions  83 non-null     int64 
dtypes: int64(2), object(2)
memory usage: 3.2+ KB
